In [1]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
import regex as re
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
with zipfile.ZipFile("C:/Users/Hp Pc/rec_datasets/movies.zip") as file:
    file_name = file.namelist()
    with file.open(file_name[0], 'r') as f:
        movies = pd.read_csv(f)
        print("Extraction Successful")


Extraction Successful


In [3]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
with zipfile.ZipFile("C:/Users/Hp Pc/rec_datasets/ratings.zip") as file:
    rate = file.open("ratings.csv")
    ratings = pd.read_csv(rate)
    print("Extaction Successful")

Extaction Successful


In [5]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414


DEVELOP A SEARCH

In [6]:
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9 ]', "", text)

In [7]:
movies['title'] = movies['title'].apply(clean_text)

In [8]:
movies['title']

0                          Toy Story 1995
1                            Jumanji 1995
2                   Grumpier Old Men 1995
3                  Waiting to Exhale 1995
4        Father of the Bride Part II 1995
                       ...               
62418                             We 2018
62419             Window of the Soul 2001
62420                      Bad Poems 2018
62421                   A Girl Thing 2001
62422         Women of Devils Island 1962
Name: title, Length: 62423, dtype: object

In [9]:
movies['tags'] = movies['title'].apply(clean_text) +" " + movies['genres'].apply(clean_text)

In [23]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words ='english')

title_vec = vectorizer.fit_transform(movies['title'])
tags_vec = vectorizer.fit_transform(movies['tags'])

In [24]:
print(vectorizer.get_feature_names_out())

['00' '00 awakening' '00 schneider' ... 'zyzzyx rd' 'zzero' 'zzero 1974']


In [ ]:
title = "Toy Story"

title = clean_text(title)
word_vec  = vectorizer.transform([title])
similarity = cosine_similarity(word_vec, title_vec)

In [27]:
similarity

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
similarity = cosine_similarity(word_vec, title_vec)

In [29]:
similarity

array([[0., 0., 0., ..., 0., 0., 0.]])

In [30]:
indices = np.argpartition(similarity, -5)[-5:]

In [31]:
indices

array([[    1, 31211, 62421, ..., 14492,  2433, 39693]], dtype=int64)

In [32]:
results = movies.iloc[indices[0]]

In [33]:
results

,movieId,title,genres,tags
1,2,Jumanji 1995,Adventure|Children|Fantasy,Jumanji 1995 AdventureChildrenFantasy
31211,138022,Man in the Mirror The Michael Jackson Story 2004,Documentary|Drama,Man in the Mirror The Michael Jackson Story 20...
62421,209169,A Girl Thing 2001,(no genres listed),A Girl Thing 2001 no genres listed
62420,209163,Bad Poems 2018,Comedy|Drama,Bad Poems 2018 ComedyDrama
62419,209159,Window of the Soul 2001,Documentary,Window of the Soul 2001 Documentary
...,...,...,...,...
19639,101962,Wolf Children Okami kodomo no ame to yuki 2012,Animation|Fantasy,Wolf Children Okami kodomo no ame to yuki 2012...
39697,157789,hack Liminality In the Case of Yuki Aihara,(no genres listed),hack Liminality In the Case of Yuki Aihara no ...
14492,76156,Yuki Nina 2009,Drama,Yuki Nina 2009 Drama
2433,2524,Towering Inferno The 1974,Action|Adventure|Drama|Thriller,Towering Inferno The 1974 ActionAdventureDrama...
